<a href="https://colab.research.google.com/github/Wizcy1121/Predictive-Analytics--Projects/blob/main/customer_perchase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project Overview
The objective is to predict the amount spent by the customer in a store in response to a mail containing campaigns catalog.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets, linear_model,preprocessing
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_absolute_error, \
mean_squared_error, r2_score, make_scorer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split, cross_val_score, \
GridSearchCV , KFold, cross_validate, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor, \
RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from lightgbm import  LGBMRegressor
from scipy.stats import reciprocal
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df= pd.read_excel("/content/drive/MyDrive/MSBA 6421/HW/purchase.xlsx", index_col=0)

In [6]:
df.head()

,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,source_r,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
sequence_number,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,1,0,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,127.87
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0.00
3,1,0,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127.48
4,1,0,1,0,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0.00
5,1,0,1,0,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0.00


In [7]:
# Creating the features dataset
X = df.iloc[:,:-1]

# Creating the target dataset
y = df.iloc[:, -1]

# Creating the train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


The objective is to predict the amount spent by the customer in a store in response to a mail containing campaigns catalog.

### Data transformation:
The whole dataset is split into training and test data with a proportion of 0.8 and 0.2 respectively. For models like K-NN, SVM, it is necessary for the features to be normalized. This is to make sure that all the attributes are present on the same scale as opposed to different scales for different attributes and making sure that no attributes are dominated by other attributes while the model is being trained. Thus the features are normalized based on Min-Max Scaling and applied to all the models for the ease of simplicity and also because it does not have any negative impact.

In [8]:
# Normalization
min_max_scaler = preprocessing.MinMaxScaler()

X_train_trans = min_max_scaler.fit_transform(X_train)
X_train_trans = pd.DataFrame(X_train_trans)
X_train_trans.columns = X_train.columns[0:23]

X_test_trans = min_max_scaler.fit_transform(X_test)
X_test_trans = pd.DataFrame(X_test_trans)
X_test_trans.columns = X_test.columns[0:23]

### Feature Selection:
ForFor this dataset, we intend to build the following models to predict the spending of a customer based on the given attributes. However, we presume that most of the variance in the target variable would be explained by just some of the independent variables. Hence, we perform feature selection on the independent variables to select the top 10 variables in the order of importance to fit the model. We then compare the accuracy of the same model with and without feature selection in order to check how much accuracy we are losing. We used Linear Regression to perform this comparison of accuracy. From the results we got, it is evident that the difference in the accuracy is so minimal between the model with and without feature selection. Since using lesser number of features reduces the complexity of model as well as computation, we used the top 10 important variables to train all the models.

In [9]:
#Feature selection using Random Forest
feat = RandomForestRegressor(n_estimators=100, random_state=42)
feat.fit(X_train_trans, y_train)

feature_importance = pd.DataFrame(np.round(feat.feature_importances_,3),
                                   index = X_train.columns,
                                columns=['importance']).sort_values('importance',
                                ascending = False)

In [10]:
#Features are ranked based on how they reduce impurity and create pure nodes.
#Based on the ranking, we decide to pick up the top 10 features
#Based on the values, we take the top 10 features and compare how performance differs.

top_features = feature_importance.nlargest(10,"importance")

#Normalizing the variables
X_train_trans_fs = min_max_scaler.fit_transform(X_train.filter(items = top_features.index))

X_test_trans_fs = min_max_scaler.transform(X_test.filter(items = top_features.index))

X_train_fs = pd.DataFrame(X_train_trans_fs)
X_test_fs = pd.DataFrame(X_test_trans_fs)

In [11]:
X_train_fs

,0,1,2,3,4,5,6,7,8,9
0,0.066667,1.0,0.573919,0.573919,0.0,0.0,1.0,0.0,0.0,0.0
1,0.066667,1.0,0.069023,0.069023,0.0,0.0,1.0,1.0,1.0,0.0
2,0.066667,1.0,0.577024,0.577024,0.0,0.0,1.0,1.0,1.0,1.0
3,0.066667,1.0,0.399331,0.399331,0.0,1.0,1.0,1.0,0.0,0.0
4,0.066667,0.0,0.696919,0.696919,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1595,0.066667,0.0,0.681872,0.681872,0.0,0.0,1.0,0.0,0.0,0.0
1596,0.000000,0.0,0.039647,0.039647,1.0,0.0,1.0,0.0,0.0,0.0
1597,0.066667,0.0,0.300931,0.300931,1.0,0.0,1.0,1.0,0.0,0.0
1598,0.000000,0.0,0.669214,0.669214,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# k-Fold initialization
scoring = 'neg_root_mean_squared_error'
inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
# outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

### Modelling Overview:
We have used 7 types of prediction models to predict the amount spent by the customer. The models are:

* Linear regression (Elastic net)
* Regression Tree 
* K-NN regression
* SVM regression
* Gradient Boosing Regression 
* Random Forest Regression
* LightGBM regression

### Elastic net linear regression

In [13]:
# Initialiazing Elastic Net and creating a reg_grid for its various hyperparameters
reg = ElasticNet()

reg_grid = {
    'l1_ratio': [.1, .5, .7, .9, .95, .99, 1],
    'alpha': [0.0125, 0.025, 0.05, .125, .25, .5, 1., 2., 4.],
    "max_iter": list(range(10000, 100000))
}

1. without feature selection

In [14]:
reg_clf = RandomizedSearchCV(estimator = reg, param_distributions=reg_grid, scoring=scoring, n_iter=20, cv = inner_cv)
reg_clf.fit(X_train_trans, y_train)
y_pred = reg_clf.best_estimator_.predict(X_test_trans)

In [21]:
print(f'R-squred: {r2_score(y_test, y_pred): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

R-squred:  0.587
Root mean squared error: 124.188


2. with feature selection

In [22]:
reg_clf_fs = RandomizedSearchCV(estimator = reg, param_distributions=reg_grid, scoring=scoring, n_iter=20, cv = inner_cv)
reg_clf_fs.fit(X_train_fs, y_train)
y_pred_fs = reg_clf_fs.best_estimator_.predict(X_test_fs)

In [24]:
print(f'R-squred: {r2_score(y_test, y_pred_fs): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.3f}')

R-squred:  0.586
Root mean squared error: 124.299


### Decision Tree

In [47]:
# Initialiazing dtree and creating a dt_grid for its various hyperparameters
dt = DecisionTreeRegressor()

dt_grid ={
    "max_depth" : list(range(5,10)), 
    "max_leaf_nodes": list(range(5,20)),
    "min_samples_leaf": list(range(5,15))
}

1. without feature selection

In [48]:
dt_clf = RandomizedSearchCV(estimator = dt, param_distributions=dt_grid, scoring=scoring, n_iter=20, cv = inner_cv)
dt_clf.fit(X_train_trans, y_train)
y_pred = dt_clf.best_estimator_.predict(X_test_trans)

In [49]:
print(f'R-squred: {r2_score(y_test, y_pred): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

R-squred:  0.638
Root mean squared error: 116.219


2. with feature selection

In [51]:
dt_clf_fs = RandomizedSearchCV(estimator = dt, param_distributions=dt_grid, scoring=scoring, n_iter=20, cv = inner_cv)
dt_clf_fs.fit(X_train_fs, y_train)
y_pred_fs = dt_clf_fs.best_estimator_.predict(X_test_fs)

In [53]:
print(f'R-squred: {r2_score(y_test, y_pred_fs): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.3f}')

R-squred:  0.600
Root mean squared error: 122.121


### KNN

In [31]:
# Initialiazing knn and creating a knn_grid for its various hyperparameters
knn = KNeighborsRegressor()

knn_grid ={
    "weights" : ["uniform", "distance"], 
    "n_neighbors": list(range(1,20))
}

1. without feature selection

In [32]:
knn_clf = RandomizedSearchCV(estimator = knn, param_distributions=knn_grid, scoring=scoring, n_iter=20, cv = inner_cv)
knn_clf.fit(X_train_trans, y_train)
y_pred = knn_clf.best_estimator_.predict(X_test_trans)

In [38]:
print(f'R-squred: {r2_score(y_test, y_pred): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

R-squred:  0.311
Root mean squared error: 160.409


In [54]:
knn_clf_fs = RandomizedSearchCV(estimator = knn, param_distributions=knn_grid, scoring=scoring, n_iter=20, cv = inner_cv)
knn_clf_fs.fit(X_train_fs, y_train)
y_pred_fs = knn_clf_fs.best_estimator_.predict(X_test_fs)

In [55]:
print(f'R-squred: {r2_score(y_test, y_pred_fs): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.3f}')

R-squred:  0.329
Root mean squared error: 158.267


### SVR

In [67]:
# Building the Support Vector Regression
svr = SVR()

# Parameter grid
svr_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
               'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
{'kernel': ['linear'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
{'kernel': ['poly'], 'degree': [2,3,4,5,6], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
 'gamma': [1e-3, 1e-4]}]

1. without feature selection

In [68]:
svr_clf = RandomizedSearchCV(estimator = svr, param_distributions=svr_grid, scoring=scoring, n_iter=20, cv = inner_cv)
svr_clf.fit(X_train_trans, y_train)
y_pred = svr_clf.best_estimator_.predict(X_test_trans)

In [69]:
print(f'R-squred: {r2_score(y_test, y_pred): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

R-squred:  0.267
Root mean squared error: 165.446


2. with feature selection

In [89]:
svr_clf_fs = RandomizedSearchCV(estimator = svr, param_distributions=svr_grid, scoring=scoring, n_iter=20, cv = inner_cv)
svr_clf_fs.fit(X_train_fs, y_train)
y_pred_fs = svr_clf_fs.best_estimator_.predict(X_test_fs)

In [90]:
print(f'R-squred: {r2_score(y_test, y_pred_fs): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.3f}')

R-squred:  0.090
Root mean squared error: 184.286


### Gradient Boost

In [58]:
# Building the GradientBoostingRegressor()
gbr = GradientBoostingRegressor()

# Parameter grid
gbr_grid = {'n_estimators' : [500],
                  'learning_rate': reciprocal(0.01, 0.1),
                  'max_depth': list(range(4,15)),
                  'min_samples_leaf': list(range(10,200)),
                  'max_features': [1.0, 0.3, 0.1]}

1. without feature selection

In [72]:
gbr_clf = RandomizedSearchCV(estimator = gbr, param_distributions=gbr_grid, scoring=scoring, n_iter=20, cv = inner_cv)
gbr_clf.fit(X_train_trans, y_train)
y_pred = gbr_clf.best_estimator_.predict(X_test_trans)

In [73]:
print(f'R-squred: {r2_score(y_test, y_pred): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

R-squred:  0.657
Root mean squared error: 113.209


2. with feature selection

In [74]:
gbr_clf_fs = RandomizedSearchCV(estimator = gbr, param_distributions=gbr_grid, scoring=scoring, n_iter=20, cv = inner_cv)
gbr_clf_fs.fit(X_train_fs, y_train)
y_pred_fs = gbr_clf_fs.best_estimator_.predict(X_test_fs)

In [75]:
print(f'R-squred: {r2_score(y_test, y_pred_fs): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.3f}')

R-squred:  0.583
Root mean squared error: 124.790


### Random Forest

In [76]:
# Building the RandomForestRegressor()
rfr = RandomForestRegressor()

# Parameter grid
rfr_grid = {'max_depth': list(range(1,20)),
            'criterion' : ["absolute_error"],
            'max_features': ['auto'],
            "bootstrap": [True, False],
            'max_depth': list(range(5, 15))
            }

1. without feature selection

In [77]:
rfr_clf = RandomizedSearchCV(estimator = rfr, param_distributions=rfr_grid, scoring=scoring, n_iter=20, cv = inner_cv)
rfr_clf.fit(X_train_trans, y_train)
y_pred = rfr_clf.best_estimator_.predict(X_test_trans)

In [78]:
print(f'R-squred: {r2_score(y_test, y_pred): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

R-squred:  0.543
Root mean squared error: 130.581


2. with feature selection

In [79]:
rfr_clf_fs = RandomizedSearchCV(estimator = rfr, param_distributions=rfr_grid, scoring=scoring, n_iter=20, cv = inner_cv)
rfr_clf_fs.fit(X_train_fs, y_train)
y_pred_fs = rfr_clf_fs.best_estimator_.predict(X_test_fs)

In [80]:
print(f'R-squred: {r2_score(y_test, y_pred_fs): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.3f}')

R-squred:  0.597
Root mean squared error: 122.709


### LightGBM

In [81]:
gbm = LGBMRegressor()

gbm_grid = {
    'num_iterations': list(range(100,1000)),
    'learning_rate': reciprocal(3e-3, 3e-1),
    'max_depth': list(range(5,30)) ,
    'n_estimators': list(range(50,300,10)),
    'num_leaves': list(range(50,400,5))
}

1. without feature selection

In [82]:
gbm_clf = RandomizedSearchCV(estimator = gbm, param_distributions=gbm_grid, scoring=scoring, n_iter=20, cv = inner_cv)
gbm_clf.fit(X_train_trans, y_train)
y_pred = gbm_clf.best_estimator_.predict(X_test_trans)

In [83]:
print(f'R-squred: {r2_score(y_test, y_pred): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

R-squred:  0.670
Root mean squared error: 111.015


2. with feature selection

In [84]:
gbm_clf_fs = RandomizedSearchCV(estimator = gbm, param_distributions=gbm_grid, scoring=scoring, n_iter=20, cv = inner_cv)
gbm_clf_fs.fit(X_train_fs, y_train)
y_pred_fs = gbm_clf_fs.best_estimator_.predict(X_test_fs)

In [85]:
print(f'R-squred: {r2_score(y_test, y_pred_fs): .3f}')
print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred_fs)):.3f}')

R-squred:  0.627
Root mean squared error: 118.052
